In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

# ------------------ SETUP ------------------
torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ------------------ DATASET ------------------
class EMGDataset(Dataset):
    def __init__(self, x_path, y_path):
        self.X = np.load(x_path)
        self.y = np.argmax(np.load(y_path), axis=1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.X[idx], dtype=torch.float32),
            torch.tensor(self.y[idx], dtype=torch.long)
        )

# ------------------ MODEL ------------------
class CNNEncoder(nn.Module):
    def __init__(self, emb_dim=128):
        super().__init__()
        self.bn = nn.BatchNorm1d(6)
        self.conv1 = nn.Conv1d(6, 128, 10)
        self.pool1 = nn.MaxPool1d(3)
        self.conv2 = nn.Conv1d(128, 256, 10)
        self.pool2 = nn.MaxPool1d(3)
        self.gap = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(256, emb_dim)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.bn(x)
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.gap(x).squeeze(-1)
        z = F.normalize(self.fc(x), dim=1)
        return z

class FullModel(nn.Module):
    def __init__(self, emb_dim=128, num_classes=62):
        super().__init__()
        self.encoder = CNNEncoder(emb_dim)
        self.classifier = nn.Linear(emb_dim, num_classes)

    def forward(self, x):
        z = self.encoder(x)
        logits = self.classifier(z)
        return z, logits

# ------------------ ALL–ALL MINING MASKS ------------------
def batch_all_masks(labels):
    labels = labels.unsqueeze(1)
    pos = labels == labels.T
    neg = labels != labels.T
    pos.fill_diagonal_(False)
    return pos, neg

# ------------------ TRIPLET LOSSES ------------------
class TripletL2(nn.Module):
    def __init__(self, margin=0.2):
        super().__init__()
        self.margin = margin

    def forward(self, z, y):
        dist = torch.cdist(z, z).pow(2)
        pos, neg = batch_all_masks(y)

        dap = (dist * pos).max(dim=1).values
        dan = (dist + 1e6 * (~neg)).min(dim=1).values

        return torch.log1p(torch.exp(dap - dan + self.margin)).mean()

class TripletCosine(nn.Module):
    def __init__(self, margin=0.1):
        super().__init__()
        self.margin = margin

    def forward(self, z, y):
        sim = z @ z.T
        pos, neg = batch_all_masks(y)

        sap = (sim * pos).max(dim=1).values
        san = (sim - 1e6 * (~neg)).min(dim=1).values

        return torch.log1p(torch.exp(san - sap + self.margin)).mean()

class NTXent(nn.Module):
    def __init__(self, temp=0.2):
        super().__init__()
        self.temp = temp

    def forward(self, z, y):
        sim = (z @ z.T) / self.temp
        pos, _ = batch_all_masks(y)

        exp_sim = torch.exp(sim)
        pos_sum = (exp_sim * pos).sum(dim=1)
        denom = exp_sim.sum(dim=1)

        return (-torch.log(pos_sum / denom + 1e-8)).mean()

# ------------------ EARLY STOPPING ------------------
class EarlyStopping:
    def __init__(self, patience=10):
        self.patience = patience
        self.best_acc = -1
        self.counter = 0
        self.best_state = None

    def step(self, acc, model):
        if acc > self.best_acc:
            self.best_acc = acc
            self.best_state = {k: v.detach().cpu() for k, v in model.state_dict().items()}
            self.counter = 0
            return False
        else:
            self.counter += 1
            return self.counter >= self.patience

    def restore(self, model):
        model.load_state_dict(self.best_state)

# ------------------ TRAIN LOOP ------------------
def train_and_eval(model, train_loader, test_loader,
                   triplet_loss, alpha=1.0,
                   epochs=50, patience=10):

    ce = nn.CrossEntropyLoss()
    opt = Adam(model.parameters(), lr=5e-4)
    stopper = EarlyStopping(patience)

    for ep in range(epochs):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)

            z, logits = model(x)
            loss = ce(logits, y) + alpha * triplet_loss(z, y)

            opt.zero_grad()
            loss.backward()
            opt.step()

        model.eval()
        correct = total = 0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                _, logits = model(x)
                pred = logits.argmax(1)
                correct += (pred == y).sum().item()
                total += y.size(0)

        acc = correct / total
        print(f"Epoch {ep+1:02d} | Val Acc: {acc:.4f}")

        if stopper.step(acc, model):
            print("Early stopping triggered")
            break

    stopper.restore(model)
    return stopper.best_acc

# ------------------ EXPERIMENT ------------------
BASE = "models/Data/Data/62_classes/UserIndependent"

losses = {
    "Triplet_L2": TripletL2(),
    "Triplet_Cosine": TripletCosine(),
    "NTXent": NTXent()
}

results = {}

for name, loss_fn in losses.items():
    print(f"\n========== {name} ==========")
    accs = []

    for split in range(1, 11):
        print(f"\n--- Split {split} ---")

        train_ds = EMGDataset(
            f"{BASE}/Train/X_train_{split}.npy",
            f"{BASE}/Train/y_train_{split}.npy"
        )
        test_ds = EMGDataset(
            f"{BASE}/Test/X_test_{split}.npy",
            f"{BASE}/Test/y_test_{split}.npy"
        )

        train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
        test_loader = DataLoader(test_ds, batch_size=64)

        model = FullModel().to(device)
        acc = train_and_eval(model, train_loader, test_loader, loss_fn)
        accs.append(acc)

        del model
        torch.cuda.empty_cache()

    results[name] = np.mean(accs)
    print(f"AVG ACC ({name}): {results[name]:.4f}")

print("\nFINAL RESULTS")
for k, v in results.items():
    print(f"{k:20s} : {v:.4f}")


Using device: cuda

========== Triplet_L2 ==========

--- Split 1 ---
Epoch 01 | Val Acc: 0.2032
Epoch 02 | Val Acc: 0.4161
Epoch 03 | Val Acc: 0.5323
Epoch 04 | Val Acc: 0.5774
Epoch 05 | Val Acc: 0.5758
Epoch 06 | Val Acc: 0.6065
Epoch 07 | Val Acc: 0.6210
Epoch 08 | Val Acc: 0.6597
Epoch 09 | Val Acc: 0.6194
Epoch 10 | Val Acc: 0.6532
Epoch 11 | Val Acc: 0.6323
Epoch 12 | Val Acc: 0.6016
Epoch 13 | Val Acc: 0.6242
Epoch 14 | Val Acc: 0.6145
Epoch 15 | Val Acc: 0.6226
Epoch 16 | Val Acc: 0.6081
Epoch 17 | Val Acc: 0.6258
Epoch 18 | Val Acc: 0.6048
Early stopping triggered

--- Split 2 ---
Epoch 01 | Val Acc: 0.0968
Epoch 02 | Val Acc: 0.2323
Epoch 03 | Val Acc: 0.3435
Epoch 04 | Val Acc: 0.3839
Epoch 05 | Val Acc: 0.4129
Epoch 06 | Val Acc: 0.4710
Epoch 07 | Val Acc: 0.4468
Epoch 08 | Val Acc: 0.4435
Epoch 09 | Val Acc: 0.4839
Epoch 10 | Val Acc: 0.5016
Epoch 11 | Val Acc: 0.4839
Epoch 12 | Val Acc: 0.4887
Epoch 13 | Val Acc: 0.4968
Epoch 14 | Val Acc: 0.4952
Epoch 15 | Val Acc: 0.48

In [2]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

from pytorch_metric_learning import losses, miners, distances

torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

class EMGDataset(Dataset):
    def __init__(self, x_path, y_path):
        self.X = np.load(x_path)
        self.y = np.argmax(np.load(y_path), axis=1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.X[idx], dtype=torch.float32),
            torch.tensor(self.y[idx], dtype=torch.long)
        )

class CNNEncoder(nn.Module):
    def __init__(self, emb_dim=256):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv1d(6, 64, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(64, 64, kernel_size=10),
            nn.ReLU(),
            nn.MaxPool1d(3),

            nn.Conv1d(64, 256, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(256, 256, kernel_size=10),
            nn.ReLU(),

            nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Linear(256, emb_dim)

    def forward(self, x):
        x = x.permute(0, 2, 1)         
        x = self.features(x).squeeze(-1)
        z = self.fc(x)
        return z                        

class FullModel(nn.Module):
    def __init__(self, emb_dim=256, num_classes=62):
        super().__init__()
        self.encoder = CNNEncoder(emb_dim)
        self.classifier = nn.Linear(emb_dim, num_classes)

    def forward(self, x):
        z = self.encoder(x)
        logits = self.classifier(z)
        return z, logits

def train_and_eval(model, train_loader, test_loader,
                   metric_loss, miner,
                   epochs=500, patience=10):

    ce_loss = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=5e-4)

    best_acc = -1
    patience_ctr = 0
    best_state = None

    for epoch in range(epochs):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)

            embeddings, logits = model(x)

            indices_tuple = miner(embeddings, y)
            loss = (
                metric_loss(embeddings, y, indices_tuple)
                + ce_loss(logits, y)
            )

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

       
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                _, logits = model(x)
                preds = logits.argmax(1)
                correct += (preds == y).sum().item()
                total += y.size(0)

        acc = correct / total
        print(f"Epoch {epoch+1:03d} | Val Acc: {acc:.4f}")

        if acc > best_acc:
            best_acc = acc
            best_state = {k: v.cpu() for k, v in model.state_dict().items()}
            patience_ctr = 0
        else:
            patience_ctr += 1
            if patience_ctr >= patience:
                print("Early stopping")
                break

    model.load_state_dict(best_state)
    return best_acc

encoder_losses = {
    "Triplet_L2": losses.TripletMarginLoss(
        margin=0.2,
        distance=distances.LpDistance(normalize_embeddings=True, p=2, power=2),
        smooth_loss=True
    ),
    "Triplet_Cosine": losses.TripletMarginLoss(
        margin=0.1,
        distance=distances.CosineSimilarity(),
        smooth_loss=True
    ),
    "NTXent": losses.NTXentLoss(temperature=0.2)
}


miner = miners.BatchEasyHardMiner(
    pos_strategy="semihard",
    neg_strategy="hard"
)

BASE = "models/Data/Data/62_classes/UserIndependent"
final_results = {}

for loss_name, metric_loss in encoder_losses.items():
    print(f"\n==============================")
    print(f" Encoder Loss: {loss_name}")
    print(f"==============================")

    accs = []

    for split in range(1, 20):
        print(f"\n----- SPLIT {split} -----")

        train_ds = EMGDataset(
            f"{BASE}/Train/X_train_{split}.npy",
            f"{BASE}/Train/y_train_{split}.npy"
        )
        test_ds = EMGDataset(
            f"{BASE}/Test/X_test_{split}.npy",
            f"{BASE}/Test/y_test_{split}.npy"
        )

        train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
        test_loader = DataLoader(test_ds, batch_size=128, shuffle=False)

        model = FullModel().to(device)

        acc = train_and_eval(
            model,
            train_loader,
            test_loader,
            metric_loss,
            miner,
            epochs=500,
            patience=10
        )

        print(f"Split Accuracy: {acc:.4f}")
        accs.append(acc)

        del model
        torch.cuda.empty_cache()

    final_results[loss_name] = np.mean(accs)
    print(f"\n>>> Average Accuracy ({loss_name}): {final_results[loss_name]:.4f}")

print("\n================ FINAL RESULTS ================")
for k, v in final_results.items():
    print(f"{k:20s} : {v:.4f}")


Using device: cuda

 Encoder Loss: Triplet_L2

----- SPLIT 1 -----
Epoch 001 | Val Acc: 0.0210
Epoch 002 | Val Acc: 0.1645
Epoch 003 | Val Acc: 0.4355
Epoch 004 | Val Acc: 0.5532
Epoch 005 | Val Acc: 0.5806
Epoch 006 | Val Acc: 0.6403
Epoch 007 | Val Acc: 0.6097
Epoch 008 | Val Acc: 0.5968
Epoch 009 | Val Acc: 0.6194
Epoch 010 | Val Acc: 0.6194
Epoch 011 | Val Acc: 0.6129
Epoch 012 | Val Acc: 0.6210
Epoch 013 | Val Acc: 0.6177
Epoch 014 | Val Acc: 0.5919
Epoch 015 | Val Acc: 0.6000
Epoch 016 | Val Acc: 0.6258
Early stopping
Split Accuracy: 0.6403

----- SPLIT 2 -----
Epoch 001 | Val Acc: 0.0306
Epoch 002 | Val Acc: 0.1000
Epoch 003 | Val Acc: 0.1177
Epoch 004 | Val Acc: 0.2823
Epoch 005 | Val Acc: 0.2984
Epoch 006 | Val Acc: 0.3032
Epoch 007 | Val Acc: 0.3742
Epoch 008 | Val Acc: 0.3597
Epoch 009 | Val Acc: 0.4161
Epoch 010 | Val Acc: 0.4081
Epoch 011 | Val Acc: 0.4129
Epoch 012 | Val Acc: 0.4484
Epoch 013 | Val Acc: 0.4403
Epoch 014 | Val Acc: 0.4468
Epoch 015 | Val Acc: 0.4048
Epoch 

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

from pytorch_metric_learning import losses, miners, distances

torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ---------------- DATASET ----------------
class EMGDataset(Dataset):
    def __init__(self, x_path, y_path):
        X = np.load(x_path)
        y = np.argmax(np.load(y_path), axis=1)

        # channel-wise normalization (CRITICAL)
        for i in range(X.shape[0]):
            for c in range(6):
                X[i, :, c] = (X[i, :, c] - X[i, :, c].mean()) / (X[i, :, c].std() + 1e-8)

        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# ---------------- CNN + BiLSTM ENCODER ----------------
class CNNBiLSTMEncoder(nn.Module):
    def __init__(self, emb_dim=256):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.Conv1d(6, 64, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(64, 64, kernel_size=10),
            nn.ReLU(),
            nn.MaxPool1d(3),

            nn.Conv1d(64, 256, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(256, 256, kernel_size=10),
            nn.ReLU()
        )

        self.bilstm = nn.LSTM(
            input_size=256,
            hidden_size=128,
            batch_first=True,
            bidirectional=True
        )

        self.fc = nn.Linear(256, emb_dim)

    def forward(self, x):
        x = x.permute(0, 2, 1)      # (B, C, T)
        x = self.cnn(x)             # (B, 256, T')
        x = x.permute(0, 2, 1)      # (B, T', 256)

        out, _ = self.bilstm(x)
        feat = out[:, -1, :]        # last timestep

        z = self.fc(feat)
        return z

# ---------------- FULL MODEL ----------------
class FullModel(nn.Module):
    def __init__(self, emb_dim=256, num_classes=62):
        super().__init__()
        self.encoder = CNNBiLSTMEncoder(emb_dim)
        self.classifier = nn.Linear(emb_dim, num_classes)

    def forward(self, x):
        z = self.encoder(x)
        logits = self.classifier(z)
        return z, logits

# ---------------- TRAIN + EVAL ----------------
def train_and_eval(model, train_loader, test_loader,
                   metric_loss, miner,
                   epochs=300, patience=12):

    ce_loss = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=3e-4)

    best_acc = -1
    patience_ctr = 0
    best_state = None

    for epoch in range(epochs):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)

            z, logits = model(x)

            # 🔥 CRITICAL FIX #1
            z = F.normalize(z, dim=1)

            # 🔥 CRITICAL FIX #2
            indices_tuple = miner(z, y)

            # 🔥 CRITICAL FIX #3 (λ = 0.1)
            loss = (
                ce_loss(logits, y)
                + 0.1 * metric_loss(z, y, indices_tuple)
            )

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

        # -------- VALIDATION --------
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                _, logits = model(x)
                preds = logits.argmax(1)
                correct += (preds == y).sum().item()
                total += y.size(0)

        acc = correct / total
        print(f"Epoch {epoch+1:03d} | Val Acc: {acc:.4f}")

        if acc > best_acc:
            best_acc = acc
            best_state = {k: v.cpu() for k, v in model.state_dict().items()}
            patience_ctr = 0
        else:
            patience_ctr += 1
            if patience_ctr >= patience:
                print("Early stopping")
                break

    model.load_state_dict(best_state)
    return best_acc

# ---------------- METRIC LOSSES ----------------
encoder_losses = {
    "Triplet_L2": losses.TripletMarginLoss(
        margin=0.2,
        distance=distances.LpDistance(p=2),
        smooth_loss=True
    ),
    "Triplet_Cosine": losses.TripletMarginLoss(
        margin=0.1,
        distance=distances.CosineSimilarity(),
        smooth_loss=True
    ),
    "NTXent": losses.NTXentLoss(temperature=0.2)
}

# 🔥 CRITICAL FIX #4 (SOFT miner)
miner = miners.BatchEasyHardMiner(
    pos_strategy="easy",
    neg_strategy="semihard"
)

BASE = "models/Data/Data/62_classes/UserIndependent"

final_results = {}

for loss_name, metric_loss in encoder_losses.items():
    print(f"\n========== {loss_name} ==========")
    accs = []

    for split in range(1, 20):
        train_ds = EMGDataset(
            f"{BASE}/Train/X_train_{split}.npy",
            f"{BASE}/Train/y_train_{split}.npy"
        )
        test_ds = EMGDataset(
            f"{BASE}/Test/X_test_{split}.npy",
            f"{BASE}/Test/y_test_{split}.npy"
        )

        train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
        test_loader = DataLoader(test_ds, batch_size=128)

        model = FullModel().to(device)

        acc = train_and_eval(
            model,
            train_loader,
            test_loader,
            metric_loss,
            miner
        )

        print(f"Split {split} Acc: {acc:.4f}")
        accs.append(acc)

        del model
        torch.cuda.empty_cache()

    final_results[loss_name] = np.mean(accs)

print("\n===== FINAL RESULTS =====")
for k, v in final_results.items():
    print(f"{k:20s}: {v:.4f}")


Using device: cuda

========== Triplet_L2 ==========
Epoch 001 | Val Acc: 0.0516
Epoch 002 | Val Acc: 0.0984
Epoch 003 | Val Acc: 0.1371
Epoch 004 | Val Acc: 0.2048
Epoch 005 | Val Acc: 0.2935
Epoch 006 | Val Acc: 0.3935
Epoch 007 | Val Acc: 0.4355
Epoch 008 | Val Acc: 0.4323
Epoch 009 | Val Acc: 0.4484
Epoch 010 | Val Acc: 0.5032
Epoch 011 | Val Acc: 0.4823
Epoch 012 | Val Acc: 0.5371
Epoch 013 | Val Acc: 0.5258
Epoch 014 | Val Acc: 0.5677
Epoch 015 | Val Acc: 0.6129
Epoch 016 | Val Acc: 0.5371
Epoch 017 | Val Acc: 0.5758
Epoch 018 | Val Acc: 0.5742
Epoch 019 | Val Acc: 0.6339
Epoch 020 | Val Acc: 0.5790
Epoch 021 | Val Acc: 0.6161
Epoch 022 | Val Acc: 0.5806
Epoch 023 | Val Acc: 0.5677
Epoch 024 | Val Acc: 0.6371
Epoch 025 | Val Acc: 0.6194
Epoch 026 | Val Acc: 0.5952
Epoch 027 | Val Acc: 0.5694
Epoch 028 | Val Acc: 0.5952
Epoch 029 | Val Acc: 0.6097
Epoch 030 | Val Acc: 0.6194
Epoch 031 | Val Acc: 0.6210
Epoch 032 | Val Acc: 0.6290
Epoch 033 | Val Acc: 0.6339
Epoch 034 | Val Acc: 0.